# Project Galilei
### "Measure what is measurable, and make measurable what is not so."

## PDF Retrieval & Processing

In [2]:
from io import BytesIO
import requests


import numpy as np
import pandas as pd

import arxivscraper as ax

import PyPDF2
import pdfplumber

from sklearn.feature_extraction.text import CountVectorizer

In [6]:
scraper = ax.Scraper(category='physics:cond-mat', date_from='2017-05-27',date_until='2017-06-07')

In [7]:
output = scraper.scrape()

fetching up to  1000 records...
fetching is completed in 12.0 seconds.
Total number of records 545


In [8]:
cols = ('id', 'title', 'categories', 'abstract', 'doi', 'created', 'updated', 'authors')
df = pd.DataFrame(output)

In [10]:
df.head()

,title,id,abstract,categories,doi,created,updated,authors,affiliation,url
0,generalized extensivity,1008.1559,in order to apply thermodynamics to systems in...,cond-mat.stat-mech,,2010-08-09,2016-02-19,"[john e. gray, stephen r. addison]",[],https://arxiv.org/abs/1008.1559
1,general relations for quantum gases in two and...,1210.1784,we derive exact general relations between vari...,cond-mat.quant-gas,10.1103/physreva.86.053633,2012-10-05,2017-05-30,"[félix werner, yvan castin]","[lkb, lkb]",https://arxiv.org/abs/1210.1784
2,distribution of lifetimes of kinetochore-micro...,1310.1579,"before a cell divides into two daughter cells,...",physics.bio-ph cond-mat.stat-mech q-bio.sc,10.1088/1478-3975/11/3/036004,2013-10-06,2014-03-24,"[ajeet k. sharma, blerta shtylla, debashish ch...",[],https://arxiv.org/abs/1310.1579
3,incorporation of a dc bias in a high-q 3d micr...,1311.4422,we report a technique for applying a dc bias i...,cond-mat.mes-hall,,2013-11-18,2017-05-29,"[mingyun yuan, sal j. bosman, g. a. steele]",[],https://arxiv.org/abs/1311.4422
4,a s=1/2 vanadium-based geometrically frustrate...,1405.1697,we report the synthesis and characterization o...,cond-mat.str-el,10.1088/0953-8984/26/26/265601,2014-05-07,2014-05-13,"[tanmoy chakrabarty, a. v. mahajan, b. koteswa...",[],https://arxiv.org/abs/1405.1697


In [4]:
arxiv_meta = pd.read_json("../data/raw/arxiv-metadata-oai-snapshot.json", lines=True, nrows=10000)

In [5]:
arxiv_meta.head(5)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"
3,704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]"
4,704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"


In [ ]:
url = "https://arxiv.org/pdf/0704.0001.pdf"
response = requests.get(url)
my_raw_data = response.content

with BytesIO(my_raw_data) as data:
    read_pdf = PyPDF2.PdfFileReader(data)
    print(read_pdf.numPages)
    df = pd.DataFrame(data = read_pdf.getPage(11).extractText(), columns=["Text"], index = [0])

#     for page in range(read_pdf.getNumPages()):
#         print(read_pdf.getPage(page).extractText())

In [ ]:
df["Text"].iloc[0]